<a href="https://colab.research.google.com/github/touseefyounas/PV_Analysis/blob/main/PVWatts_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import data_table
import requests
import pandas as pd
import plotly.express as px
import folium
from folium.raster_layers import ImageOverlay
import plotly.graph_objects as go


In [3]:
url="https://developer.nrel.gov/api/pvwatts/v8"
api_key="Rrm78PdXJt41jDkniLFvy5M2NyFY90cxQd2xGopJ"
latitude=28.5384
longitude=-81.3789
params={'format':'json',
        'api_key':api_key,
        'system_capacity':10,
        'module_type':0,
        'losses':14,
        'array_type':1,
        'tilt':30,
        'azimuth':180,
        'lat':latitude,
        'lon':longitude,
        'albedo':0.3,
        'inv_eff':94}

response=requests.get(f"{url}.json",params=params)

if response.status_code==200:
  pv_data=response.json()

  #ac_monthly=pv_data['outputs']['ac_monthly']
  output_data=pv_data['outputs']
else:
  print(f"Error: {response.status_code} - {response.text}")

Months=['January','February','March','April','May','June','July','August','September','October','November','December']

#ac_pd=pd.DataFrame(table_data)
output_df=pd.DataFrame(output_data)
output_df.insert(0,'Month',Months)
output_df.head(12)

,Month,ac_monthly,poa_monthly,solrad_monthly,dc_monthly,ac_annual,solrad_annual,capacity_factor
0,January,1242.089162,162.888378,5.254464,1329.252449,15220.591388,5.662066,17.375104
1,February,1121.065586,148.239338,5.294262,1199.770034,15220.591388,5.662066,17.375104
2,March,1449.822502,194.065990,6.260193,1549.830575,15220.591388,5.662066,17.375104
3,April,1411.559471,192.550877,6.418363,1510.543458,15220.591388,5.662066,17.375104
4,May,1415.387101,195.265213,6.298878,1515.253391,15220.591388,5.662066,17.375104
5,June,1181.094225,167.335103,5.577837,1266.641177,15220.591388,5.662066,17.375104
6,July,1217.956547,171.613974,5.535935,1306.228647,15220.591388,5.662066,17.375104
7,August,1270.666189,178.926277,5.771815,1361.991528,15220.591388,5.662066,17.375104
8,September,1220.995086,168.206972,5.606899,1307.811318,15220.591388,5.662066,17.375104
9,October,1369.440419,183.434046,5.917227,1464.431076,15220.591388,5.662066,17.375104


In [4]:
total_dc=round(output_df['dc_monthly'].sum(),0)

total_sun=round(output_df['solrad_annual'].mean()*365,0)

fig = go.Figure()

fig.add_trace(go.Indicator(
    mode = 'number',
    value = total_dc,
    number = {'suffix': ' kwh','valueformat':'0'},
    title = {'text': 'Annual Power Generation Potential'},
    delta = {'position': 'top'},#, 'reference': 320},
    domain = {'x': [0, 0.5], 'y': [0, 1]}))

fig.add_trace(go.Indicator(
    mode = 'number',
    value = total_sun,
    number = {'suffix': ' hours','valueformat':'0'},
    title = {'text': 'Useable Sunlight Available per Year'},
    delta = {'position': 'top'},#, 'reference': 320},
    domain = {'x': [0.5, 1], 'y': [0, 1]})
    )

fig.update_layout(paper_bgcolor = '#18191A',font_color='#FFFFFF')

fig.show()

In [5]:
fig = px.bar(output_df, x='Month', y='solrad_monthly',
       hover_data='solrad_monthly', color='dc_monthly',
       labels={'solrad_monthly':'Monthly Average of Daily Solar Hours (h)'},color_continuous_scale=['yellow','orange','red'],opacity=0.9)
fig.update_layout(paper_bgcolor='#18191A',plot_bgcolor='#242526',font_color='#FFFFFF',coloraxis_colorbar_title_text = 'Monthly DC array output (kWhdc)',bargap=0.7)
fig.show()

In [6]:
# Create a base map
test_map = folium.Map(location=[latitude,longitude ], zoom_start=17)  # Adjust latitude and longitude as needed

# Add markers, popups, or other map features
folium.Marker([latitude, longitude], popup='Your Location').add_to(test_map)
folium.TileLayer('cartodbdark_matter').add_to(test_map)
folium.LayerControl().add_to(test_map)
# Display the map in a Jupyter Notebook or save it as an HTML file
test_map.save('map.html')  # Save as an HTML file
test_map


In [7]:
url="https://developer.nrel.gov/api/pvwatts/v8"
api_key="Rrm78PdXJt41jDkniLFvy5M2NyFY90cxQd2xGopJ"
params2={'format':'json',
        'api_key':api_key,
        'system_capacity':4,
        'module_type':0,
        'losses':14,
        'array_type':1,
        'tilt':30,
        'azimuth':180,
        'lat':28.5384,
        'lon':-81.3789,
        'timeframe':'hourly',
        'albedo':0.3,
        'inv_eff':94}

hourly_response=requests.get(f"{url}.json",params=params2)

if hourly_response.status_code==200:
  pv_hourly_data=hourly_response.json()

  #ac_monthly=pv_data['outputs']['ac_monthly']
else:
  print(f"Error: {response.status_code} - {response.text}")

hourly_dn=pd.DataFrame(pv_hourly_data['outputs']['ac_monthly'])
hourly_dn.head(5)

,0
0,496.835665
1,448.426234
2,579.929001
3,564.623788
4,566.154840


In [8]:
pv_hourly_data['outputs'].keys()

dict_keys(['ac_monthly', 'poa_monthly', 'solrad_monthly', 'dc_monthly', 'ac_annual', 'solrad_annual', 'capacity_factor', 'ac', 'poa', 'dn', 'dc', 'df', 'tamb', 'tcell', 'wspd', 'alb'])

In [9]:
hourly_df=pd.DataFrame({'ac':pv_hourly_data['outputs']['ac'],
                        'poa':pv_hourly_data['outputs']['poa'],
                        'dn':pv_hourly_data['outputs']['dn'],
                        'dc':pv_hourly_data['outputs']['dc'],
                        'df':pv_hourly_data['outputs']['df'],
                        'tamb':pv_hourly_data['outputs']['tamb'],
                        'tcell':pv_hourly_data['outputs']['tcell'],
                        'wspd':pv_hourly_data['outputs']['wspd'],
                        'alb':pv_hourly_data['outputs']['alb']})
hourly_df.head(5)

,ac,poa,dn,dc,df,tamb,tcell,wspd,alb
0,0.0,0.0,0.0,0.0,0.0,15.0,15.0,1.3,0.12
1,0.0,0.0,0.0,0.0,0.0,15.0,15.0,1.3,0.12
2,0.0,0.0,0.0,0.0,0.0,15.0,15.0,1.3,0.12
3,0.0,0.0,0.0,0.0,0.0,14.0,14.0,1.3,0.12
4,0.0,0.0,0.0,0.0,0.0,14.0,14.0,1.3,0.12


In [10]:
for index,_ in enumerate(hourly_df.index):
  hour=(index % 24) + 1
  julian_day=(index // 24) + 1
  week=(index // 168) + 1
  hourly_df.at[index,'hour']=hour
  hourly_df.at[index,'julianDay']=julian_day
  hourly_df.at[index,'week']=week
hourly_df.head(5)

,ac,poa,dn,dc,df,tamb,tcell,wspd,alb,hour,julianDay,week
0,0.0,0.0,0.0,0.0,0.0,15.0,15.0,1.3,0.12,1.0,1.0,1.0
1,0.0,0.0,0.0,0.0,0.0,15.0,15.0,1.3,0.12,2.0,1.0,1.0
2,0.0,0.0,0.0,0.0,0.0,15.0,15.0,1.3,0.12,3.0,1.0,1.0
3,0.0,0.0,0.0,0.0,0.0,14.0,14.0,1.3,0.12,4.0,1.0,1.0
4,0.0,0.0,0.0,0.0,0.0,14.0,14.0,1.3,0.12,5.0,1.0,1.0


In [11]:
weekly_temp_pivot = hourly_df.pivot_table(index='week', columns='hour', values='tamb')
fig = px.imshow(weekly_temp_pivot.T, color_continuous_scale='RdBu_r', origin='lower')
fig.update_layout(height=600,paper_bgcolor='#18191A',plot_bgcolor='#242526',font_color='#FFFFFF',coloraxis_colorbar_title_text = 'Temperature (C)')
fig.show()

In [12]:
hourly_temp_pivot=hourly_df.pivot_table(index='julianDay', columns='hour', values='tamb')
# Define the x, y, and z data for the 3D surface plot

x = hourly_temp_pivot.columns  # Hour of the day
y = hourly_temp_pivot.index  # Day of the year
z = hourly_temp_pivot.values  # Temperature values

fig = go.Figure(data=[go.Surface(z=z, x=x, y=y)])

# Customize the 3D plot layout
fig.update_layout(scene=dict(aspectmode="manual", aspectratio=dict(x=1, y=1, z=0.5)),scene_xaxis_title='Hour of the Day',
    scene_yaxis_title='Day of the Year',scene_zaxis_title='Temperature (C)',paper_bgcolor='#18191A',font_color='#FFFFFF',
    coloraxis_colorbar_title_text='Temparature (C)')
fig.update_layout(scene = dict(xaxis = dict(backgroundcolor="rgba(0, 0, 0,0)",gridcolor="white",showbackground=True,zerolinecolor="white",),
                               yaxis = dict(backgroundcolor="rgba(0, 0, 0,0)",gridcolor="white",showbackground=True,zerolinecolor="white"),
                               zaxis = dict(backgroundcolor="rgba(0, 0, 0,0)",gridcolor="white",showbackground=True,zerolinecolor="white")))
fig.update_xaxes(range=[0, 24], title_text='Hour of the Day')
fig.update_yaxes(range=[0, 365], title_text='Day of the Year')
fig.show()


In [28]:
cost_df=output_df
utility_rate=residential_rate
cost_df['monthly_bill']=round(utility_rate*cost_df['ac_monthly'],2)
cost_df['cumulative_bill']=cost_df['monthly_bill'].cumsum()
cost_df.head(12)

,Month,ac_monthly,poa_monthly,solrad_monthly,dc_monthly,ac_annual,solrad_annual,capacity_factor,monthly_bill,cumulative_bill
0,January,1242.089162,162.888378,5.254464,1329.252449,15220.591388,5.662066,17.375104,138.00,138.00
1,February,1121.065586,148.239338,5.294262,1199.770034,15220.591388,5.662066,17.375104,124.55,262.55
2,March,1449.822502,194.065990,6.260193,1549.830575,15220.591388,5.662066,17.375104,161.08,423.63
3,April,1411.559471,192.550877,6.418363,1510.543458,15220.591388,5.662066,17.375104,156.82,580.45
4,May,1415.387101,195.265213,6.298878,1515.253391,15220.591388,5.662066,17.375104,157.25,737.70
5,June,1181.094225,167.335103,5.577837,1266.641177,15220.591388,5.662066,17.375104,131.22,868.92
6,July,1217.956547,171.613974,5.535935,1306.228647,15220.591388,5.662066,17.375104,135.31,1004.23
7,August,1270.666189,178.926277,5.771815,1361.991528,15220.591388,5.662066,17.375104,141.17,1145.40
8,September,1220.995086,168.206972,5.606899,1307.811318,15220.591388,5.662066,17.375104,135.65,1281.05
9,October,1369.440419,183.434046,5.917227,1464.431076,15220.591388,5.662066,17.375104,152.14,1433.19


In [29]:
fig = px.bar(cost_df, x='Month', y='monthly_bill',
       hover_data='ac_monthly', color='ac_monthly',
       labels={'value':'Monthly Value (CAD$)'},color_continuous_scale=['yellow','orange','red'],opacity=0.9)
fig.update_layout(paper_bgcolor='#18191A',plot_bgcolor='#242526',font_color='#FFFFFF',coloraxis_colorbar_title_text = 'Monthly AC system output (kWhac)',bargap=0.7)
fig.show()

In [30]:
fig = px.area(cost_df, x='Month', y='monthly_bill',
       hover_data='ac_monthly',
       labels={'value':'Monthly Value (CAD$)'})
fig.update_layout(paper_bgcolor='#18191A',plot_bgcolor='#242526',font_color='#FFFFFF')
fig.update_traces(line=dict(color='orange', width=3))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(range=[0, max(cost_df['monthly_bill'])])
fig.show()

In [31]:
fig = px.area(cost_df, x='Month', y='cumulative_bill',hover_data='cumulative_bill',labels={'cumulative_bill':'Total Value (CAD$)'})
fig.update_layout(paper_bgcolor='#18191A',plot_bgcolor='#242526',font_color='#FFFFFF',coloraxis_showscale=False)
fig.update_traces(fillcolor='rgba(245,161,39,0.3)')
fig.update_traces(line=dict(color='yellow'))
fig.update_xaxes(showgrid=False)
fig.show()

In [17]:
pv_hourly_data['outputs'].keys()

dict_keys(['ac_monthly', 'poa_monthly', 'solrad_monthly', 'dc_monthly', 'ac_annual', 'solrad_annual', 'capacity_factor', 'ac', 'poa', 'dn', 'dc', 'df', 'tamb', 'tcell', 'wspd', 'alb'])

In [18]:
test_monthly=pd.DataFrame({'ac_monthly':pv_hourly_data['outputs']['ac_monthly'],
                        'poa_monthly':pv_hourly_data['outputs']['poa_monthly'],
                        'solrad_monthly':pv_hourly_data['outputs']['solrad_monthly'],
                        'dc_monthly':pv_hourly_data['outputs']['dc_monthly']})
test_monthly.head(5)

,ac_monthly,poa_monthly,solrad_monthly,dc_monthly
0,496.835665,162.888378,5.254464,531.700980
1,448.426234,148.239338,5.294262,479.908014
2,579.929001,194.065990,6.260193,619.932230
3,564.623788,192.550877,6.418363,604.217383
4,566.154840,195.265213,6.298878,606.101356


In [19]:
hourly_poa_pivot = hourly_df.pivot_table(index='julianDay', columns='hour', values='poa')

x = hourly_poa_pivot.columns  # Hour of the day
y = hourly_poa_pivot.index  # Day of the year
z = hourly_poa_pivot.values  # Temperature values

fig = go.Figure(data=[go.Surface(z=z, x=x, y=y)])

# Customize the 3D plot layout
fig.update_layout(scene=dict(aspectmode="manual", aspectratio=dict(x=1, y=1, z=0.5)),scene_xaxis_title='Hour of the Day',
    scene_yaxis_title='Day of the Year',scene_zaxis_title='Hourly plane of array irradiance (W/m2)',paper_bgcolor='#18191A',font_color='#FFFFFF',
    coloraxis_colorbar_title_text='Temparature (C)')
fig.update_layout(scene = dict(xaxis = dict(backgroundcolor="rgba(0, 0, 0,0)",gridcolor="white",showbackground=True,zerolinecolor="white",),
                               yaxis = dict(backgroundcolor="rgba(0, 0, 0,0)",gridcolor="white",showbackground=True,zerolinecolor="white"),
                               zaxis = dict(backgroundcolor="rgba(0, 0, 0,0)",gridcolor="white",showbackground=True,zerolinecolor="white")))
fig.update_xaxes(range=[0, 24], title_text='Hour of the Day')
fig.update_yaxes(range=[0, 365], title_text='Day of the Year')
fig.show()

In [20]:
hourly_dff = hourly_df.groupby('julianDay').mean()

# Add a column for Date in the table
hourly_dff['date'] = pd.to_datetime(hourly_dff.index, unit='D', origin=pd.Timestamp('2022-12-31'))

# Add a column for months in the table
hourly_dff['month'] = hourly_dff['date'].dt.strftime('%B')
hourly_dff.head(5)

,ac,poa,dn,dc,df,tamb,tcell,wspd,alb,hour,week,date,month
julianDay,,,,,,,,,,,,,
1.0,707.869667,235.708958,234.250000,757.095833,37.250,18.458333,24.688917,1.904167,0.12,12.5,1.0,2023-01-01,January
2.0,536.713375,181.566917,131.416667,575.592708,64.000,19.250000,25.508750,1.345833,0.12,12.5,1.0,2023-01-02,January
3.0,657.276833,220.187792,221.416667,703.523000,41.000,20.000000,25.849250,1.733333,0.12,12.5,1.0,2023-01-03,January
4.0,697.284833,232.924167,235.375000,745.887583,43.000,20.625000,26.709500,1.987500,0.12,12.5,1.0,2023-01-04,January
5.0,717.258750,237.589292,236.625000,767.037875,45.125,19.625000,25.366708,1.879167,0.12,12.5,1.0,2023-01-05,January


In [50]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=hourly_dff.index, y=hourly_dff['tamb'],
   # hoverinfo='x+y',
    mode='lines',
    line=dict(width=0.5, color='rgba(245,161,39,0.5)'),
    stackgroup='one' # define stack group
))
fig.update_layout(paper_bgcolor='#18191A',plot_bgcolor='#242526',font_color='#FFFFFF')
fig.update_traces(line=dict(color='orange', width=3))
fig.update_xaxes(showgrid=False)
fig.show()

In [52]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=hourly_dff.index, y=hourly_dff['dn'],
   # hoverinfo='x+y',
    mode='lines',
    line=dict(width=2, color='rgba(245,161,39,0.5)'),
    stackgroup='one' # define stack group
))
fig.add_trace(go.Scatter(
    x=hourly_dff.index, y=hourly_dff['df'],
   # hoverinfo='x+y',
    mode='lines',
    line=dict(width=2, color='rgba(30, 130, 76, 0.5)'),
    stackgroup='one' # define stack group
))
fig.update_layout(paper_bgcolor='#18191A',plot_bgcolor='#242526',font_color='#FFFFFF',
                  showlegend=False,)
                  #margin=dict(l=20, r=20, b=10, t=20, pad=1),)

#fig.update_traces(line=dict(color='orange', width=3))
fig.update_xaxes(showgrid=False)
fig.show()

In [53]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=hourly_dff.index, y=hourly_dff['wspd'],
   # hoverinfo='x+y',
    mode='lines',
    line=dict(width=2, color='rgba(30, 130, 76, 0.5)'),
    stackgroup='one' # define stack group
))
fig.update_layout(paper_bgcolor='#18191A',plot_bgcolor='#242526',font_color='#FFFFFF',
                  showlegend=False,)
                  #margin=dict(l=20, r=20, b=10, t=20, pad=1),)

#fig.update_traces(line=dict(color='orange', width=3))
fig.update_xaxes(showgrid=False)
fig.show()

In [23]:
url_openei="https://api.openei.org/utility_rates"
api_utility="xsn2kIrHm3aNAaLWf0XqmvJ8NWATgxSP1TJ0nvFu"
params_openei={
    'version':'latest',
    'format':'json',
    'api_key':api_utility,
    'limit':1,
    'detail':'full',
    'lat':28.5384,
    'lon':-81.3789
                }

utility_openei=requests.get(url_openei,params=params_openei)

if utility_openei.status_code==200:
  openei_data=utility_openei.json()
else:
  print(f"Error: {utility_openei.status_code} - {utility_openei.text}")

openei_data.keys()

dict_keys(['items'])

In [24]:
url="https://developer.nrel.gov/api/utility_rates/v3"
api_key="Rrm78PdXJt41jDkniLFvy5M2NyFY90cxQd2xGopJ"
params_v3={'format':'json',
        'api_key':api_key,
        'lat':28.5384,
        'lon':-81.3789
         }

rate_response=requests.get(f"{url}.json",params=params_v3)

if rate_response.status_code==200:
  rate_data=rate_response.json()

else:
  print(f"Error: {response.status_code} - {response.text}")

residential_rate=rate_data['outputs']['residential']
industrial_rate=rate_data['outputs']['industrial']
commercial_rate=rate_data['outputs']['commercial']

# print(f'Residential Rate: {residential_rate} $/kwh')
# print(f'Industrial Rate: {industrial_rate} $/kwh')
# print(f'Commercial Rate: {commercial_rate} $/kwh')



In [25]:
def get_user_input():
    print("Please select an option:")
    print("1. Enter monthly bill value")
    print("2. Enter average bill value")
    print("3. Enter monthly consumption value")
    print("4. Enter average consumption value")
    choice = input("Your choice (1-4): ")

    value = None

    if choice == '1':
        value = input("Enter your monthly bill value ($): ")
        # Process the monthly bill value
        print(f"You entered a monthly bill value of {value}$")
    elif choice == '2':
        value = input("Enter your average bill value ($): ")
        # Process the average bill value
        print(f"You entered an average bill value of {value}$")
    elif choice == '3':
        value = input("Enter your monthly consumption value (kwh): ")
        # Process the monthly consumption value
        print(f"You entered a monthly consumption value of {value} kwh")
    elif choice == '4':
        value = input("Enter your average consumption value (kwh): ")
        # Process the average consumption value
        print(f"You entered an average consumption value of {value} kwh")
    else:
        print("Invalid choice. Please enter a number between 1 and 4.")

    return value

# Call the function and assign its return value to a variable
user_input_value = get_user_input()
print(user_input_value)


Please select an option:
1. Enter monthly bill value
2. Enter average bill value
3. Enter monthly consumption value
4. Enter average consumption value
Your choice (1-4): 2
Enter your average bill value ($): 130
You entered an average bill value of 130$
130


In [26]:
# Example data
categories = ['January','February','March','April','May','June','July','August','September','October','November','December']
actual_values = [70, 50, 55, 50, 60, 70, 90, 45, 80, 95, 85, 75]  # Actual values

# Max value for the filling tube
if sum(actual_values)==0:
  max_value=100
else:
  max_value = max(actual_values)


# Create two bars for each category: one for the actual value, one for the unfilled portion
fig = go.Figure()

for category, actual in zip(categories, actual_values):
    # Bar for the actual value
    fig.add_trace(go.Bar(
        name=category,
        x=[category], y=[actual],
        marker_color='black',
        hovertemplate='$%{y}'
    ))

    # Bar for the unfilled portion
    fig.add_trace(go.Bar(
        name='',
        x=[category], y=[max_value - actual],
        marker_color='lightgrey',
        hoverinfo='none'  # This disables hover info for the unfilled part
    ))


# Update layout
fig.update_layout(
    barmode='stack',  # This stacks the two parts of each bar on top of each other
    title_text=None,
    yaxis=dict(title='Value',showgrid=False),
    xaxis=dict(title=None,showgrid=False),
    bargap=0.775,
    showlegend=False,
    margin=dict(l=20, r=20, b=10, t=20, pad=1),
    )

fig.show()

In [27]:
for key, value in pv_hourly_data['outputs'].items():
  if isinstance(value,list):
    print(f"Length of {key}: {len(value)}")
  else:
    print(f'{key} has no len()')

Length of ac_monthly: 12
Length of poa_monthly: 12
Length of solrad_monthly: 12
Length of dc_monthly: 12
ac_annual has no len()
solrad_annual has no len()
capacity_factor has no len()
Length of ac: 8760
Length of poa: 8760
Length of dn: 8760
Length of dc: 8760
Length of df: 8760
Length of tamb: 8760
Length of tcell: 8760
Length of wspd: 8760
Length of alb: 8760
